# Setup

In [7]:
!nvidia-smi
!pip install "ultralytics>=8.3.50" roboflow==1.*

Wed Oct 22 10:30:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             33W /   70W |     838MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!rm -rf /content/sample_data/

# Load Dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="UGuKv3GDa6PNiTMeOoLd")
project = rf.workspace("workspaceendeavor360").project("formula-student-cones-roz6f") #personal workspace
version = project.version(1)
dataset = version.download("yolov12")
print("Dataset downloaded to:", dataset.location)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Formula-Student-Cones-1 in yolov12:: 100%|██████████| 4268/4268 [00:00<00:00, 6649.08it/s]

✅ Dataset downloaded to: /content/Formula-Student-Cones-1


# To save after Disconnect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/cone-detector-yolov12-1024"
!mkdir -p "$PROJECT_DIR"


Mounted at /content/drive


# Now train on the custom exported dataset

In [5]:
from ultralytics import YOLO
import os
os.chdir(dataset.location)

# Load base model
model = YOLO("yolov8n.pt")
# Start training
results = model.train(
    data="data.yaml",
    epochs=50,
    imgsz=1024,
    batch=8,
    device=0,
    rect=True,
    project=PROJECT_DIR,
    name="cones_yolov12_1024"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.219 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_w

# Test

In [ ]:
os.chdir("/content")

%rm -rf /content/pred_custom

import os, cv2
from ultralytics import YOLO

best_path = "/content/drive/MyDrive/cone-detector-yolov12-1024/cones_yolov12_1024/weights/best.pt"
model = YOLO(best_path)

test_path = "/content/Formula-Student-Cones-1/test/images/"

results = model.predict(source=test_path, conf=0.5, save=False)

out_dir = "/content/pred_custom"
os.makedirs(out_dir, exist_ok=True)

for i, r in enumerate(results):
    img = r.plot(labels=True, conf=True, line_width=1, font_size=0.2)
    cv2.imwrite(os.path.join(out_dir, f"{i:04d}.jpg"), img)

print("Custom predictions saved to:", out_dir)



image 1/213 /content/Formula-Student-Cones-1/test/images/00000005_png_jpg.rf.d5bac08fbcfdc991058a8ce14cf95fb1.jpg: 1024x1024 4 big oranges, 11 blue cones, 15 yellow cones, 14.4ms
image 2/213 /content/Formula-Student-Cones-1/test/images/00000025_png_jpg.rf.de08b777531cd004b4d7433f3421f263.jpg: 1024x1024 4 big oranges, 15 blue cones, 14 yellow cones, 13.4ms
image 3/213 /content/Formula-Student-Cones-1/test/images/00000120_png_jpg.rf.e9675e2ce6b1a533c7461292238d25bd.jpg: 1024x1024 10 blue cones, 8 yellow cones, 13.3ms
image 4/213 /content/Formula-Student-Cones-1/test/images/00000150_png_jpg.rf.476992bf8cc9b9ff2f13793447a20acd.jpg: 1024x1024 11 blue cones, 10 yellow cones, 13.4ms
image 5/213 /content/Formula-Student-Cones-1/test/images/00000200_png_jpg.rf.77cc27ae89ee4e395964794da55616e4.jpg: 1024x1024 11 blue cones, 13 yellow cones, 13.3ms
image 6/213 /content/Formula-Student-Cones-1/test/images/00000325_png_jpg.rf.e8c2cdf1e2f47e077875b0ab24e47378.jpg: 1024x1024 3 big oranges, 8 blue con

In [25]:
os.chdir(dataset.location)
metrics = model.val(data="data.yaml", split="test", imgsz=1024, batch=8, device=0)
print(metrics)

Ultralytics 8.3.219 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1757.9±495.7 MB/s, size: 54.0 KB)
val: Scanning /content/Formula-Student-Cones-1/test/labels... 213 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 213/213 1.8Kit/s 0.1s
val: New cache created: /content/Formula-Student-Cones-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 27/27 3.7it/s 7.3s
                   all        213       8597      0.897      0.677      0.783      0.512
            big orange         36        105      0.839      0.829      0.873      0.543
             blue cone        213       4149      0.906      0.575      0.702       0.46
           yellow cone        201       4343      0.947      0.629      0.774      0.532
Speed: 6.8ms preprocess, 10.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /content/Formula-Student-Cones-1/ru